In [2]:
import findspark
findspark.init()

In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()

In [38]:
df1 = spark.read.parquet("D://TechnoAvengers//DI//Spark_Optimizations//data//parquet//employee.parquet")
df1.createOrReplaceTempView("t1")
df1.show()

+---+-------+
| id|  first|
+---+-------+
|  1| Torrey|
|  2|  Micah|
|  3| Hollis|
|  4|  Perry|
|  5|Janelle|
|  6|Charity|
|  7|  Dejah|
|  8|  Ellen|
|  9|   Sven|
| 10|Ryleigh|
| 11| Cooper|
| 12|Esteban|
| 13|Lucinda|
| 14| Jarvis|
| 15| Jordon|
| 16|Marques|
| 17|  Edgar|
| 18|  Adell|
| 19|Madonna|
| 20|  Andre|
+---+-------+
only showing top 20 rows



In [39]:
df2 = spark.read.parquet("D://TechnoAvengers//DI//Spark_Optimizations//data//parquet//country.parquet")
df2.createOrReplaceTempView("t2")
df2.show()

+---+--------------------+
| id|             country|
+---+--------------------+
|  1|         Switzerland|
|  2|Democratic People...|
|  3|             Tunisia|
|  4|                Chad|
|  5|           Swaziland|
|  6|             Lebanon|
|  7|               Egypt|
|  8|              Israel|
|  9|               Macao|
| 10|               Congo|
| 11|             Reunion|
| 12|          Guadeloupe|
| 13|             Lebanon|
| 14|               Ghana|
| 15|              Serbia|
| 16|               Sudan|
| 17|   Wallis and Futuna|
| 18|        Sierra Leone|
| 19|             Namibia|
| 20|             Georgia|
+---+--------------------+
only showing top 20 rows



In [41]:
df3 = spark.sql("select * from t1 join t2 on t1.id==t2.id where t2.country!='Switzerland'")

In [42]:
df3.explain(mode="extended")

== Parsed Logical Plan ==
'Project [*]
+- 'Filter NOT ('t2.country = Switzerland)
   +- 'Join Inner, ('t1.id = 't2.id)
      :- 'UnresolvedRelation [t1]
      +- 'UnresolvedRelation [t2]

== Analyzed Logical Plan ==
id: bigint, first: string, id: bigint, country: string
Project [id#337L, first#338, id#357L, country#358]
+- Filter NOT (country#358 = Switzerland)
   +- Join Inner, (id#337L = id#357L)
      :- SubqueryAlias t1
      :  +- Relation[id#337L,first#338] parquet
      +- SubqueryAlias t2
         +- Relation[id#357L,country#358] parquet

== Optimized Logical Plan ==
Join Inner, (id#337L = id#357L)
:- Filter isnotnull(id#337L)
:  +- Relation[id#337L,first#338] parquet
+- Filter ((isnotnull(country#358) AND NOT (country#358 = Switzerland)) AND isnotnull(id#357L))
   +- Relation[id#357L,country#358] parquet

== Physical Plan ==
*(2) BroadcastHashJoin [id#337L], [id#357L], Inner, BuildRight
:- *(2) Project [id#337L, first#338]
:  +- *(2) Filter isnotnull(id#337L)
:     +- *(2) Col

In [43]:
df3.explain(mode="formatted")

== Physical Plan ==
* BroadcastHashJoin Inner BuildRight (10)
:- * Project (4)
:  +- * Filter (3)
:     +- * ColumnarToRow (2)
:        +- Scan parquet  (1)
+- BroadcastExchange (9)
   +- * Project (8)
      +- * Filter (7)
         +- * ColumnarToRow (6)
            +- Scan parquet  (5)


(1) Scan parquet 
Output [2]: [id#337L, first#338]
Batched: true
Location: InMemoryFileIndex [file:/D:/TechnoAvengers/DI/Spark_Optimizations/data/parquet/employee.parquet]
PushedFilters: [IsNotNull(id)]
ReadSchema: struct<id:bigint,first:string>

(2) ColumnarToRow [codegen id : 2]
Input [2]: [id#337L, first#338]

(3) Filter [codegen id : 2]
Input [2]: [id#337L, first#338]
Condition : isnotnull(id#337L)

(4) Project [codegen id : 2]
Output [2]: [id#337L, first#338]
Input [2]: [id#337L, first#338]

(5) Scan parquet 
Output [2]: [id#357L, country#358]
Batched: true
Location: InMemoryFileIndex [file:/D:/TechnoAvengers/DI/Spark_Optimizations/data/parquet/country.parquet]
PushedFilters: [IsNotNull(country)